![図1.2 Sparkの紹介とインストール](images/PysparkInProduction.png)

# 本セクションの目次

1. 本番で動かすことを前提とした、これまでと本レクチャーの違い
2. Pysparkを本番環境で動かす際の流れ
3. Spark Submitを動かす(Sparkをコマンドラインで実行する方法)
4. チューニングのコツ
   1. Spark Webインタフェースを用いたボトルネックの調査
   2. メモリへの登録

# Sparkを用いたバッチにおけるデータエンジニアリング一連の流れ(復習)

1. データソースの読み込み(今回は、人口統計データ(/dataset/jinkou.csv))　ETL(Extract Transform Load)で言うEの部分
2. 変換を行う(集計等を行う)　DataFrame処理 or SQL処理の２パターンで実行可能 ETL(Extract Transform Load)で言うTの部分
3. カラムナーフォーマットへ変換する ETL(Extract Transform Load)で言うTの部分
4. 出力したデータをみんなに見やすくするため(BIツールから参照できるように)テーブルを作成する ETL(Extract Transform Load)で言うLの部分

本チャプターではこの流れを、Spark-Submitを使った方法で実行してみようと思います。
ただし、テーブルは事前に作成しておくことにします。

# 本番で動かすことを前提とした、これまでと本セクションの違い
これまでは、ローカル環境での開発を想定していました。  
本レクチャーでは、本番環境で実行をするための準備を行っていきます。  

セクション3,4で行ってきた内容を、本番環境に適用することを考えていきます。

主に使うリソースは./pysaprk_batch/spark_etlフォルダ配下に入っています。  

# 各リソースの説明
- etl.sql(sparkSQL。データの変換を担当するSQL。セクション4で紹介した処理と同じだが、元号だけあえて可変引数にしている。spark_etl_sample.pyで読み込んで実行する)
- jinko_schema.json(jinko.csvのスキーマ定義が入っている。動的に行うためにはファイルで外出しすることが好ましい。spark_etl_sample.pyで読み込んで適用する)
- spark_etl_sample.py(データエンジニアリング一覧の流れが記載されている)

# 環境の作成

もしPython3がインストールされていない場合は、以下からインストールをしましょう
https://www.python.org/downloads/release/python-381/

1. ターミナルを開きます。
2. インストールしたらpip3 install pysparkを実行しましょう
3. spark-submitのコマンドが出てくれば成功です

# PySparkを実際に本番環境で動かす際の流れ
ここからのメインのトピックとしては、

- スケジューラーで実行するために、spark_etl_sample.pyをコマンドベース(ターミナルを本番環境と考えてくだし)でどのように動かすか？
- 本番で直面する問題や、チューニングする際に利用するツールやそのコツ

を紹介していきます。

注。　ここからNoteBookに移動したり、はたまたターミナルに移動したりします。ご了承ください

# metastore_dbのロックを外します

削除するもの
- metasotre_db/db.lck
- metastore_db/dbex.lck

ローカル環境では、複数の環境(ノートブック、ターミナル)からテーブル定義は共有して利用できても、同時に接続することができません。  
そのため、ノートブックのロックを解除して上げる必要があります(metastore_db限定なのでMySQLやAmazon Glue Data Catalogなどはこのようなことをする必要ありません)

# spark-submit でSparkを実行する

コマンドラインでSparkを実行するにはこの方法しかない。
構文としては

```
spark-submit [メモリ設定のオプションなど] spark_etl_sample.py 引数 
```

で実行可能です。

# spark_etl_sample.pyを実行していきます
# ここからはノートブックではなくpythonエディター（ターミナル）へ移ります。

ターミナルを起動したらmetastore_dbが直下にあるところまで移動をしてください。
メタデータストアなので、ノートブックから作ったテーブル定義(jinko_avg)を利用することが可能だからです(今回だとjinko_avgのテーブル定義を使います)。

```
spark-submit ./spark_etl/spark_etl_sample.py -a hoge -s ./spark_etl/etl.sql -b GENGO=昭和 -t jinko -z ./spark_etl/jinko_schema.json -f /Users/yuki/pyspark_batch/dataset/jinko.csv
```

In [ ]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
spark = SparkSession.builder \
    .appName("chapter5") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .enableHiveSupport() \
    .getOrCreate()

In [ ]:
spark.sql("select * from jinko_avg").show()

実際にクラウド上の実行環境でspark-submitを行う場合は、実行の設定値がラップされていることが多いです。
あまり実行において気にすることはあまりない  
オンプレの場合は、細かな設定を行わなければならず苦労するポイントかも知れません。

# チューニングTips

Sparkというと、メモリに気をとられドライバー(Sparkの実行を指示する親玉)のメモリーやエグゼキューター(実行部隊)のメモリについてのチューニング記事がたくさん出てきます。  
しかしながら、これらのメモリー設定がどうしても必要になるパターンはまれです(昔はよく設定していた)。

メモリー設定を弄りまわす前に確認したいいくつかのポイントを紹介します

1. Spark web インタフェースでのボトルネックの確認
2. メモリへの登録
3. repartition数を大きくしてみる
4. Executor(ノード)の台数を増やす(クラウドであれば容易)　メモリ設定などですごく迷うのであれば、さっとノード追加して動かしてしまったほうが結局安上がり


※Sparkのプログラムはエグゼキューターと呼ばれる環境内で実行されます。Sparkのプログラムが実行されるとドライバーがエグゼギューターを立ち上げ実行をエグゼキューターに指示しエグゼキューターが実行を行います。  
今回はドライバーとエグゼキューターはローカル端末に同居しています。複数端末の場合はエグゼキューターが様々な端末で起動します。



# Spark web インタフェースでのボトルネックの確認
Sparkのウェブインタフェースを眺めてみましょう

1. sparksessionを起動する
2. http://localhost:4040 へアクセスしま

まずは簡単な画面説明から。

いくつかの処理を実行して、結果を見比べてみましょう。

In [ ]:
from pyspark.sql.types import LongType, StructType, StructField, StringType
from pyspark.sql.functions import col

# カラム名、型、デフォルト値で設定していきます
struct = StructType([
    StructField("code", StringType(), False),
    StructField("kenmei", StringType(), False),
    StructField("gengo", StringType(), False),
    StructField("wareki", StringType(), False),
    StructField("seireki", StringType(), False),
    StructField("chu", StringType(), False),
    StructField("sokei", StringType(), False),
    StructField("jinko_male", StringType(), False),
    StructField("jinko_female", StringType(), False)
])
df=spark.read.option("multiLine", "true") \
    .option("encoding", "SJIS").csv("/Users/yuki/pyspark_batch/dataset/jinko.csv", header=False, sep=',', inferSchema=False,schema=struct)

In [ ]:
df.count()

In [ ]:
spark.sql(""" 

select * from jinko_avg

""").show()

In [ ]:
spark.sql(""" 

select * from jinko_avg where kenmei='三重県'

""").show()

In [ ]:
spark.sql(""" 

select * from jinko_avg where male_avg > 3000000    

""").show()

# メモリへの登録
処理途中のデータはメモリへ登録することが可能です。  
何度も使い回す場合で比較的小さいデータはメモリ登録することで速度の向上が見込めます。

メモリの登録にはいくつか種類があります

1. dataframeをキャッシュする
2. (テンポラリ)テーブルをキャッシュする


※メモリ以外にも、ディスクに書き出すという方法もあります(persist()と呼ばれるもの、MEMORY_AND_DISKやDISK_ONLYといった形でキャッシュが可能。persist(StorageLevel.MEMORY_AND_DISK)といった書き方をする)。  
※チェックポイントと呼ばれる機能もありますが、相当長いアプリケーションでないと特徴を活かせません  


In [ ]:
# dataframeをキャッシュする場合
df=spark.sql(""" 

select * from jinko_avg where kenmei='三重県'

""")

# キャッシュする
df.cache()
# キャッシュ判定
df.is_cached

# 後続でdfを使うときはキャッシュから利用される
df.show()



In [ ]:
#キャッシュのクリア
df.unpersist()

df.is_cached

In [ ]:
# テーブルをキャッシュすることも可能です
spark.catalog.cacheTable("jinko_avg")

# キャッシュの有無
spark.catalog.isCached("jinko_avg")



In [ ]:
# キャッシュのクリア
spark.catalog.uncacheTable("jinko_avg")

# キャッシュの有無
spark.catalog.isCached("jinko_avg")

In [ ]:
# Spark利用の停止
spark.stop()
spark.sparkContext.stop()

# 演習テスト1の回答

```
spark-submit ./spark_etl/spark_etl_sample.py -a hoge -s ./spark_etl/etl.sql -b GENGO=大正 -t jinko -z ./spark_etl/jinko_schema.json -f /Users/yuki/pyspark_batch/dataset/jinko.csv
```

# 演習テスト2の回答


In [ ]:
spark.catalog.uncacheTable("jinko_avg")
spark.catalog.isCached("jinko_avg")

if not spark.catalog.isCached("jinko_avg"):
  spark.catalog.cacheTable("jinko_avg")
  print(spark.sql("select * from jinko_avg").count())